<a href="https://colab.research.google.com/github/Remorse-Code/RAG-model/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/awsdocs/aws-lambda-developer-guide.git

In [ ]:
!pip install \
  langchain_community \
  langchain_pinecone \
  langchain_openai \
  unstructured \
  langchain-text-splitters

!pip install -qU langchain-pinecone pinecone-notebooks

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob

In [ ]:
loader = DirectoryLoader('aws-lambda-developer-guide', glob="**/*.md")
documents = loader.load()

In [ ]:
documents[0]

In [ ]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('openai_api_key')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-small"
)

index_name = "pinecone-chatbot"

#splitting docs into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=20
)
split_docs = text_splitter.split_documents(documents)

In [ ]:
split_docs[25]

Document(metadata={'source': 'aws-lambda-developer-guide/sample-apps/efs-nodejs/README.md', 'text': 'Test\n\nTo invoke the function with a test event, use the invoke script.\n\nefs-nodejs$ ./4-invoke.sh\n{\n    "StatusCode": 200,\n    "ExecutedVersion": "$LATEST"\n}\n{"writeTimeMs":3.316225,"readTimeMs":166.129772,"fileSizeBytes":1398104}\n\nLet the script invoke the function a few times and then press CRTL+C to exit.\n\nCleanup\n\nTo delete the application, run the cleanup script.\n\nefs-nodejs$ ./5-cleanup.sh'}, page_content='Test\n\nTo invoke the function with a test event, use the invoke script.\n\nefs-nodejs$ ./4-invoke.sh\n{\n    "StatusCode": 200,\n    "ExecutedVersion": "$LATEST"\n}\n{"writeTimeMs":3.316225,"readTimeMs":166.129772,"fileSizeBytes":1398104}\n\nLet the script invoke the function a few times and then press CRTL+C to exit.\n\nCleanup\n\nTo delete the application, run the cleanup script.\n\nefs-nodejs$ ./5-cleanup.sh')

In [ ]:
vectorstore = PineconeVectorStore.from_documents(
    documents=split_docs,
    embedding=embeddings,
    index_name=index_name
)

In [ ]:
query = "What is the X-Ray"

similar_docs = vectorstore.similarity_search(query)
similar_docs

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever()
)

qa_chain({"query": query})